<a href="https://colab.research.google.com/github/mounilanka/Dogs-and-cats-classification/blob/main/Dogs_and_cats_Image_Classification_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset *Information

In [ ]:
#The training archive contains 25,000 images of dogs and cats .Train your algorithm on these files and predict the labels for test1.zip
#(1=dog, 0 = cat)

##Download Dataset





In [ ]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

Unzip the Dataset

In [ ]:
!unzip  kagglecatsanddogs_5340.zip

Import Modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
import tqdm
import random
from keras.preprocessing.image import load_img
warnings.filterwarnings('ignore')

Create Dataframe for Input and Output

In [ ]:
input_path = []
label = []

for class_name in os.listdir("PetImages"):
    for path in os.listdir("PetImages/"+class_name):
        if class_name == 'Cat':
            label.append(0)
        else:
            label.append(1)
        input_path.append(os.path.join("PetImages",class_name,path))
print(input_path[0],label[0])

In [ ]:
print(input_path[10],label[10])

In [ ]:
len(input_path)

In [ ]:
df = pd.DataFrame()
df['images'] = input_path
df['label'] = label
df = df.sample(frac=1).reset_index(drop=True) #we will sufill the data(data wil show in arange manager)
df.head()

In [ ]:
#delete db files
for i in df['images']:
  if '.jpg' not in i:
    print(i)


In [ ]:
import PIL
l = []
for image in df['images']:
    try:
        img = PIL.Image.open(image)
    except:
      l.append(image)
l

In [ ]:
df = df[df['images']!='PetImages/Dog/Thumbs.db']
df = df[df['images']!='PetImages/Cat/Thumbs.db']
df = df[df['images']!='PetImages/Cat/666.jpg']
df = df[df['images']!='PetImages/Dog/11702.jpg']
len(df)

Exploratory Data Analysis

In [ ]:
#to display grid of images
plt.figure(figsize=(25,25))
temp = df[df['label']==1]['images']
start = random.randint(0, len(temp))
files = temp[start:start+25]

for index,file in enumerate(files):
     plt.subplot(5,5,index+1)
     img = load_img(file)
     imp = np.array(img)
     plt.imshow(img)
     plt.title('Dogs')
     plt.axis('off')

In [ ]:
#to display grid of images
plt.figure(figsize=(25,25))
temp = df[df['label']==0]['images']
start = random.randint(0, len(temp))
files = temp[start:start+25]

for index,file in enumerate(files):
     plt.subplot(5,5,index+1)
     img = load_img(file)
     imp = np.array(img)
     plt.imshow(img)
     plt.title('cats')
     plt.axis('off')

In [ ]:
import seaborn as sns
sns.countplot(df['label'])

Create DataGenerators for the Images

In [ ]:
df['label'] = df['label'].astype('str')

In [ ]:
df.head()

In [ ]:
#input split
from sklearn.model_selection import train_test_split
train, test = train_test_split(df,test_size=0.2, random_state=42)

In [ ]:
train.head()

In [ ]:
test.head() #we have separate datas

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
    rescale = 1./255, #normalization of images
    rotation_range = 40, #augmention of images to avoid overfitting
    shear_range =0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

val_generator =ImageDataGenerator(rescale = 1./255)

train_iterator = train_generator.flow_from_dataframe(
    train,
    x_col='images',
    y_col='label',
    target_size=(128,128),
    batch_size=512,
    class_mode='binary'
)

val_iterator = val_generator.flow_from_dataframe(
    test,
    x_col='images',
    y_col='label',
    target_size=(128,128),
    batch_size=512,
    class_mode='binary'
)

In [ ]:
from keras import Sequential
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense

Model Creation

In [ ]:
from keras import Sequential
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense

model = Sequential([
                    Conv2D(16, (3,3),activation='relu',input_shape=(128,128,3)),
                    MaxPool2D((2,2)),
                    Conv2D(32, (3,3), activation ='relu'),
                    MaxPool2D((2,2)),
                    Conv2D(64, (3,3), activation='relu'),
                    MaxPool2D((2,2)),
                    Flatten(),
                    Dense(512, activation='relu'),
                    Dense(1, activation='sigmoid')

])


In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:

history = model.fit(train_iterator,epochs=10, validation_data=val_iterator)

Visualization of Results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))

plt.plot(epochs,acc,'b',label='Training Accuracy')
plt.plot(epochs,val_acc,'r',label='Validation Accuracy')
plt.title('Accuracy Graph')
plt.legend()
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(epochs,loss,'b',label='Training Loss')
plt.plot(epochs,val_loss,'r',label='Validation Loss')
plt.title('Loss Graph')
plt.legend()
plt.figure()
